# Descrição

Este notebook tem como objetivo obter estatísticas referentes ao perfil da população na força de trabalho, como seu rendimento, distribuição por raça e escolaridade, e analisar como esse público se comportou ao longo do tempo.

Autor: Isaías Rocha;
Linkedin: https://www.linkedin.com/in/isaias-santos-7ab972196/

In [ ]:
# Instalação dos pacotes
install.packages("PNADcIBGE")
install.packages("survey")
install.packages("tidyverse")
install.packages("dplyr")
install.packages("tibble")
install.packages("googledrive")
install.packages("readxl")
install.packages("rio")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘minqa’, ‘numDeriv’, ‘mitools’, ‘Rcpp’, ‘RcppArmadillo’, ‘projmgr’, ‘RCurl’, ‘survey’, ‘timeDate’




In [ ]:
# Carregando os pacotes
library(PNADcIBGE)
library(survey)
library(tidyverse)
library(dplyr)
library(tibble)
library(googledrive)
library(readxl)

rm(list = ls(all.names = TRUE)) #will clear all objects includes hidden objects.
gc()

# Baixando Micro dados da PNADc

em  *vars* seleciono apenas as variáveis de interesse (Que respondam nossa pergunta)

In [ ]:
dadosPNADc <- get_pnadc(year=2023,
                        quarter=4,
                        vars=c("UF", "V2007", "VD4007", "VD4001", "V2009",
                               "V2010", "VD4020", "V4013", "VD3004"))

# apenas pessoas na força de trabalho

dadosPNADc <- subset(dadosPNADc, VD4001 == "Pessoas na força de trabalho")
dadosPNADc <- subset(dadosPNADc, V2009 >= "14")

Média do Rendimento Das Pessoas Na Força de Trabalho

In [ ]:
mediaRendaUF = svyby(~VD4020, ~UF, dadosPNADc, svymean, na.rm=T)
rio::export(mediaRendaUF, file = "MédiaRendimentoDasPessoasNaForçadeTrabalho.xlsx")
mediaRendaUF

## Raça

V2010	10	Cor ou raça
1	Branca
2	Preta
3	Amarela
4	Parda
5	Indígena
9	Ignorado


In [ ]:
totalracaUF <- (dadosPNADct
totalracaUF <- svytotal(x=~interaction(V2010, UF), design=totalracaUF, na.rm=TRUE) %>% as.data.frame()
# Transformar o índice em campo
totalracaUF <- totalracaUF %>%
  rownames_to_column(var = "ID")

totalracaUF$ID <- gsub("interaction\\(V2010, UF\\)", "", totalracaUF$ID)

# Separar o campo inicial por "."
totalracaUF <- separate(totalracaUF, col = "ID", into = c("Raça", "UF"), sep = "\\.") %>% select(-SE)

totalracaUF <- totalracaUF %>%
  pivot_wider(names_from = `Raça`, values_from = total)

totalracaUF$Total <- totalracaUF$Branca+totalracaUF$Preta+
  totalracaUF$Amarela+totalracaUF$Parda+totalracaUF$Indígena+totalracaUF$Ignorado

totalracaUF$PartPreto <- totalracaUF$Preta/(totalracaUF$Total)*100
totalracaUF$PartPreto <- totalracaUF$PartPreto %>%
  round(digits = 2)


totalracaUF$PartParda <- totalracaUF$Parda/(totalracaUF$Total)*100
totalracaUF$PartParda <- totalracaUF$PartParda %>%
  round(digits = 2)


totalracaUF$PartIndigena <- totalracaUF$Indígena/(totalracaUF$Total)*100
totalracaUF$PartIndigena <- totalracaUF$PartIndigena %>%
  round(digits = 2)


totalracaUF

rio::export(totalracaUF, file = "EmpreendedorasPorRaça.xlsx")

## Brasil

# VD3004 – Nível de instrução mais elevado alcançado (pessoas de 5 anos ou mais de idade)

In [ ]:
# Apenas MA
MA <- subset(dadosPNADct, UF == "Maranhão")

MA <- svytotal(x=~interaction(V2007, VD3004), design=MA, na.rm=TRUE) %>% as.data.frame()

MA <- MA %>%
  rownames_to_column(var = "ID") %>% select(-c(SE))


In [ ]:
MA
rio::export(MA, file = "MA - Escolaridade_PPP.xlsx")